In [25]:
# Importing Libraries

import pandas as pd
import json
from pandas.io.json import json_normalize

In [28]:
# Loading JSON File as a String

json_file = json.load((open('data/world_bank_projects.json')))

In [42]:
# Loading json as Pandas DataFrame, we print the top row to see the structure of the DataFrame

json_df = pd.read_json('data/world_bank_projects.json')
print(json_df.head(1))

                                    _id  approvalfy board_approval_month  \
0  {'$oid': '52b213b38594d8a2be17c780'}        1999             November   

      boardapprovaldate                                 borrower  \
0  2013-11-12T00:00:00Z  FEDERAL DEMOCRATIC REPUBLIC OF ETHIOPIA   

            closingdate                              country_namecode  \
0  2018-07-07T00:00:00Z  Federal Democratic Republic of Ethiopia!$!ET   

  countrycode                              countryname countryshortname  ...  \
0          ET  Federal Democratic Republic of Ethiopia         Ethiopia  ...   

    sectorcode source  status  supplementprojectflg  \
0  ET,BS,ES,EP   IBRD  Active                     N   

                                          theme1  \
0  {'Percent': 100, 'Name': 'Education for all'}   

                                  theme_namecode themecode   totalamt  \
0  [{'code': '65', 'name': 'Education for all'}]        65  130000000   

  totalcommamt                         

In [8]:
#  Extract top 10 Countries
# Exercise Problem 1 Answer

top_ten = json_df['countryname'].value_counts().head(10)
print(top_ten)

Republic of Indonesia              19
People's Republic of China         19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Africa                             11
Republic of Mozambique             11
Name: countryname, dtype: int64


In [15]:
# Here, I noticed that Africa is included in this top 10 list, which is incorrect, since Africa is not a country.
# We therefore create a new output while excluding 'Africa'

top_ten_without_africa = json_df[json_df['countryname'] != 'Africa']['countryname'].value_counts().head(10)
print(top_ten_without_africa)

Republic of Indonesia              19
People's Republic of China         19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
People's Republic of Bangladesh    12
Kingdom of Morocco                 12
Republic of Mozambique             11
Islamic Republic of Pakistan        9
Name: countryname, dtype: int64


In [44]:
# Problem 2: Extracting top 10 major project themes
# First we need to take a look at the 'mjnamecode' column as per the exercise.

namecode = json_df.mjtheme_namecode
print(namecode.head())

0    [{'code': '8', 'name': 'Human development'}, {...
1    [{'code': '1', 'name': 'Economic management'},...
2    [{'code': '5', 'name': 'Trade and integration'...
3    [{'code': '7', 'name': 'Social dev/gender/incl...
4    [{'code': '5', 'name': 'Trade and integration'...
Name: mjtheme_namecode, dtype: object


In [29]:
# We can see that namecodes are repeated and that codes correspond to a 'name', and some names are blank.
# We normalize the json to produce a much cleaner format.

theme_normalized = json_normalize(json_file, 'mjtheme_namecode', ['id'])
print(theme_normalized.head(10))

  code                                          name       id
0    8                             Human development  P129828
1   11                                                P129828
2    1                           Economic management  P144674
3    6         Social protection and risk management  P144674
4    5                         Trade and integration  P145310
5    2                      Public sector governance  P145310
6   11  Environment and natural resources management  P145310
7    6         Social protection and risk management  P145310
8    7                   Social dev/gender/inclusion  P144665
9    7                   Social dev/gender/inclusion  P144665


In [33]:
# In order to find the top 10 project themes, we use value_counts again to give us the count of all the names

top_namecodes = theme_normalized['name'].value_counts().head(10)
print(top_namecodes)

Environment and natural resources management    223
Rural development                               202
Human development                               197
Public sector governance                        184
Social protection and risk management           158
Financial and private sector development        130
                                                122
Social dev/gender/inclusion                     119
Trade and integration                            72
Urban development                                47
Name: name, dtype: int64


In [40]:
# Problem 3: We have 122 blank values that we need to remove.
# We can do this by iterating over each row in theme_normalized.

# Let us create a dicitonary with appropriate values:

codes = { 0:'',
    1:'Economic management', 
    2:'Public sector governance', 
    3:'Rule of law',
    4:'Financial and private sector development', 
    5:'Trade and integration', 
    6:'Social protection and risk management', 
    7:'Social dev/gender/inclusion', 
    8:'Human development', 
    9:'Urban development', 
    10:'Rural development', 
    11:'Environment and natural resources management'
}

# Here we iterate over the generator for theme and call the dictionary of the name codes to fill in the blank values.

theme_clean = theme_normalized
for index,row in theme_clean.iterrows():
    if row['name'] == '':
        row['name'] = codes[int(row['code'])]

In [41]:
# Now lets check again how many values each topic has:

top_namecodes_clean = theme_clean['name'].value_counts().head(10)
print(top_namecodes_clean)

Environment and natural resources management    250
Rural development                               216
Human development                               210
Public sector governance                        199
Social protection and risk management           168
Financial and private sector development        146
Social dev/gender/inclusion                     130
Trade and integration                            77
Urban development                                50
Economic management                              38
Name: name, dtype: int64
